<a href="https://colab.research.google.com/github/beinghorizontal/deepspeech/blob/main/DeepSpeech_train_a_model%2C_alex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Alex speech-to-text model; Training
## Using DeepSpeech and my own data

*Trying to use my own custom wav files and train the model using the article written by an Indian guy with his own custom indian accent dataset*

##Training Your Own Model
*Taken from the [DeepSpeech docs - Training Your Own Model](https://deepspeech.readthedocs.io/en/v0.7.4/TRAINING.html#training-your-own-model)*

## Prevent random disconnects

This cell runs JS code to automatic reconnect to runtime.

In [ ]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")

#Gpu type

In [ ]:
!nvidia-smi

In [ ]:
import sys
! sudo apt-get install git-lfs

### Get the training code and checkpoints

Then clone the DeepSpeech repository and run `git lfs pull`.

In [ ]:
! git clone --branch v0.9.3 https://github.com/mozilla/DeepSpeech
!wget https://github.com/mozilla/DeepSpeech/releases/download/v0.9.3/deepspeech-0.9.3-checkpoint.tar.gz
#%mkdir /content/model_checkpoints
!tar -xvf /content/deepspeech-0.9.3-checkpoint.tar.gz

In [ ]:
%cd /content/DeepSpeech
! git lfs pull

In [ ]:
%cd /content/DeepSpeech
!pip3 install --upgrade pip==20.2.2 wheel==0.34.2 setuptools==49.6.0
!pip3 install --upgrade -e .


In [ ]:
y!pip3 uninstall tensorflow
!pip3 install 'tensorflow-gpu==1.15.4'

In [ ]:
!wget https://github.com/git-lfs/git-lfs/releases/download/v2.11.0/git-lfs-linux-amd64-v2.11.0.tar.gz
!tar xvf /content/git-lfs-linux-amd64-v2.11.0.tar.gz -C /content
!sudo bash /content/install.sh
%cd /content/DeepSpeech
!git-lfs pull

In [ ]:
!wget https://github.com/mozilla/DeepSpeech/releases/download/v0.9.3/ds_ctcdecoder-0.9.3-cp37-cp37m-manylinux1_x86_64.whl
!pip install /content/DeepSpeech/ds_ctcdecoder-0.9.3-cp37-cp37m-manylinux1_x86_64.whl

Once this command completes successfully, the environment will be ready to be activated.

## Get pre trained checkpoints from Drive 



In [ ]:
#@title Import from Drive

Mode = "mycheckpoints" #@param ["speech", "mycheckpoints", "data_dst", "models"]
Archive_name = "mycheckpoints.zip" #@param ["Scripts_t2.zip", "mycheckpoints.zip"]

#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive')

def zip_and_copy(path, mode):
  unzip_cmd=" -q "+Archive_name
  
  %cd $path
  copy_cmd = "/content/drive/My\ Drive/"+Archive_name+" "+path
  !cp $copy_cmd
  !unzip $unzip_cmd    
  !rm $Archive_name

zip_and_copy("/content", "nl")
  
print("Done!")



## Get raw data from Drive 


In [ ]:
#@title Import from Drive

Mode = "speech" #@param ["speech", "data_src", "data_dst", "data_src aligned", "data_dst aligned", "models"]
Archive_name = "Scripts_t2.zip" #@param {type:"string"}

#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive')

def zip_and_copy(path, mode):
  unzip_cmd=" -q "+Archive_name
  
  %cd $path
  copy_cmd = "/content/drive/My\ Drive/"+Archive_name+" "+path
  !cp $copy_cmd
  !unzip $unzip_cmd    
  !rm $Archive_name

zip_and_copy("/content", "nl")
  
print("Done!")



# Create csv in deepspeech readable format
## install num2words

In [ ]:
! pip3 install num2words

In [ ]:
import os
# import string
# import csv
# import argparse
import librosa                  # pip install librosa==0.7.2
import num2words                # pip install num2words
import re
import pandas as pd

def replace_func(text):
    """Remove extra characters from the transcript which are not in DeepSpeech's alphabet.txt
    """

    for ch in ['\\', '`', '‘', '’', '*', '_', ',', '"', '{', '}', '[', ']', '(', ')', '>', '#', '+', '-', '.', '!', '$', ':', ';', '|', '~', '@', '*', '<', '?', '/']:
        if ch in text:
            text = text.replace(ch, "")
        elif ch == '&':
            text = text.replace(ch, "and")

    return text


def get_audio_info(file_name):
    """Return specified audio file duration and sample rate
    """

    return librosa.get_duration(filename=file_name), librosa.get_samplerate(file_name)


# parser = argparse.ArgumentParser(description="Pre Process File")
# parser.add_argument('--wav', required=True,
#                     help='WAV Folder')
# parser.add_argument('--meta', required=True,
#                     help='Path to file with metadata')
# args = parser.parse_args()

# File containing audio file name and transcript
wav_file =  '/content/wave_data/Recording_'
trans_file = '/content/trans_data/Recording_'
# in_file = open(os.path.join(args.meta, 'txt'), 'r')

# Create target CSV file to write metadata info as per DeepSpeech requirements
# Define a writer object to write rows to file

# out_file = open(os.path.join(os.getcwd(), 'output.csv'), 'a', newline='')
# csv_writer = csv.writer(out_file)

# All CSV files must contain the following as the first line. Only run once
# df = pd.DataFrame({'wav_filename':[], 'wav_filesize':[], 'transcript':[]})
# df.to_csv('main.csv',index=False)
# csv_writer.writerow(['wav_filename', 'wav_filesize', 'transcript'])

# Keep track of total audio files and files not added to CSV due to them being too long or invalid sample rate
total_count = 0
row_count = 0
wav_l =os.listdir('/content/wave_data')
trans_l = os.listdir('/content/trans_data')

try:
    df_list = []

    for line in range(len(wav_l)):
        # line = 6
        total_count += 1

        try:
            # fname, ftext, _ = line.split("\"")

            # Separate file name and transcript from metadata file. Preprocess transcript and get audio info too
            # convert all numbers to text using num2words
            # fname = fname.strip()[1:].strip() + '.wav'
            with open(trans_file+str(line)+'.txt', 'r') as w:
                ftext = w.read()
            # print(ftext)


            ftext = ftext.strip().lower()
            ftext = replace_func(ftext).replace("  ", " ").strip()
            ftext = replace_func(ftext).replace("\n", ". ").strip()
            ftext = replace_func(ftext).replace("&","and")
            ftext = replace_func(ftext).replace(".","")
            ftext = replace_func(ftext).replace(" . ","")
            ftext = replace_func(ftext).replace(". . ",".")
            ftext = replace_func(ftext).replace(" .","")

            ftext = re.sub(r"(\d+)", lambda x: num2words.num2words(int(x.group(0))), ftext)

            wav_filename = wav_file+str(line)+'.wav'
            fdur, fsr = get_audio_info(wav_filename)

            # Don't add files which don't fit into model specifications
            # Either not 48kHz or longer than 30 secs
            if fsr != 48000:
                print("Different SR:", wav_filename)
                continue
            if fdur > 60:
                print("Too Long:", wav_filename)
                continue
            if ftext == '':
                print("No Transcript found")
                continue

            # Write each row to CSV with size info
            fsize = os.path.getsize(wav_filename)
            df_small = pd.DataFrame([wav_filename, fsize, ftext]).transpose()
            df_list.append(df_small)

            print(wav_filename, fsize, ftext)
            # csv_writer.writerow([wav_filename, fsize, ftext])
            row_count += 1
        except Exception as e:
            print(str(e))

except Exception as e:
    print(str(e))

print("Added Rows:", row_count)
print("Total Rows:", total_count, "\n")
df_final = pd.concat(df_list)
col_list = ['wav_filename', 'wav_filesize', 'transcript']
df_final.columns = col_list
df_final.to_csv('output.csv',index=False)
# in_file.close()
# out_file.close()


# if __name__ == "__main__":
#     main()


## Split output csv to train, test, dev files 

In [ ]:
import numpy as np
import pandas as pd

file = "/content/output.csv"

df1 = pd.read_csv(file)
df1['split'] = np.random.randn(df1.shape[0], 1)
# Split ratio for training set
msk = np.random.rand(len(df1)) <= 0.6
train = df1[msk]
inter = df1[~msk]
train.to_csv('/content/train.csv', index=False)
inter.to_csv('/content/intermediate.csv', index=False)

df2 = pd.read_csv('/content/intermediate.csv')
df2['split'] = np.random.randn(df2.shape[0], 1)
# Split ratio for dev and test
msk = np.random.rand(len(df2)) <= 0.5
dev = df2[msk]
test = df2[~msk]
dev.to_csv('/content/dev.csv', index=False)
test.to_csv('/content/test.csv', index=False)


Providing a filter alphabet is optional. It will exclude all samples whose transcripts contain characters not in the specified alphabet. Running the importer with -h will show you some additional options.

In [ ]:
#%cd /content/DeepSpeech

#!python3 /content/DeepSpeech/training/deepspeech_training/util/check_characters.py -alpha -unicode -csv /content/train.csv
#!python3 /content/DeepSpeech/training/deepspeech_training/util/check_characters.py -csv /content/dev.csv -alpha
#!python3 /content/DeepSpeech/training/deepspeech_training/util/check_characters.py -csv /content/test.csv -alpha

## Remove folders

In [ ]:
#!rm -rf "/content/mycheckpoints"

## Training

In [ ]:
%mkdir /content/models

In [ ]:
%cd /content/DeepSpeech/
! python3 DeepSpeech.py \
  --train_files /content/train.csv \
  --dev_files /content/dev.csv \
  --test_files /content/test.csv \
  --n_hidden 2048 \
  --checkpoint_dir /content/mycheckpoints \
  --learning_rate 0.0001 \
  --train_cudnn True \
  --train_batch_size 1 \
  --test_batch_size 1 \
  --epochs 300 \
  --max_to_keep 1 \
  --audio_sample_rate 48000 \
  --export_file_name 'ft_model' \
  --export_dir /content/models/

## Export new checkpoints to drive


In [ ]:
print("Removing old data in mycheckpoints which is not useful before export to drive")
import os

root = '/content/mycheckpoints'

best_checkpath = root+'/best_dev_checkpoint'
checkpath = root +'/checkpoint'

root_files = os.listdir(root)

with open(best_checkpath, 'r')as n:
    bestnames_file = n.read().splitlines()

best_dev_files = []
match_text = bestnames_file[0].split('best_dev-')[1]
match_text = match_text.replace('"','')
list2 = ['best_dev_checkpoint','checkpoint','flags.txt']
not_matching = [s for s in root_files if not match_text in s]

#remove files in checkpoint i.e. train data
with open(checkpath, 'r')as m:
    names_file = m.read().splitlines()

best_dev_files = []
match_text2 = names_file[0].split('train-')[1]
match_text3 = match_text2.replace('"','')

not_matching2 = [t for t in not_matching if not match_text3 in t]

new_list = list(set(not_matching2).difference(list2))

for remove_file in new_list:
    print(root+'/'+remove_file)
    os.remove(root+'/'+remove_file)

print("old data removed, now google drive transfer initiated")

from google.colab import drive
Archive_name = "mycheckpoints.zip"

drive.mount('/content/drive', force_remount=True)
def zip_and_copy(path, mode):
  zip_cmd="-r -q "+Archive_name+" "
  %cd $path
  zip_cmd+=mode
  !zip $zip_cmd
  copy_cmd = " "+Archive_name+"  /content/drive/My\ Drive/"
  !cp $copy_cmd
  !rm $Archive_name


zip_and_copy("/content", "mycheckpoints")


## Create the model for inference and export to drive

In [ ]:
%cd /content/DeepSpeech/
!python3 util/taskcluster.py --source tensorflow --artifact convert_graphdef_memmapped_format --branch r1.15 --target .
!./convert_graphdef_memmapped_format --in_graph=/content/models/ft_model.pb --out_graph=/content/models/ft_model.pbmm
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/models
!cp ft_model.pbmm /content/drive/MyDrive


## It ends here No need to explore further unless you want to make scorer (lang model)
Might useful in future though


###Create a Language Model / Scorer
*Taken from [DeepSpeech documentation - External scorer scripts](https://deepspeech.readthedocs.io/en/v0.7.4/Scorer.html)*

The LibriSpeech LM training text used by our scorer is around 4GB uncompressed, which should give an idea of the size of a corpus needed for a reasonable language model for general speech recognition. For more constrained use cases with smaller vocabularies, you don’t need as much data, but you should still try to gather as much as you can.

*We're using data from the [Europarl corpus](https://www.statmt.org/europarl/) for our Dutch scorer. This was the largest Dutch language dataset already formatted that I could find.*

In [ ]:
%cd /content/DeepSpeech/data/lm
! wget https://www.statmt.org/europarl/v7/nl-en.tgz
! tar -xzvf nl-en.tgz

Build [KenLM](https://github.com/kpu/kenlm) first.

In [ ]:
%cd /content
!wget -O - https://kheafield.com/code/kenlm.tar.gz |tar xz
%mkdir -p /content/kenlm/build


In [ ]:
%cd /content/kenlm/build
! cmake ..

In [ ]:
! make -j 4

Then use the `generate_lm.py` script to generate `lm.binary` and `vocab-500000.txt`.

As input you can use a plain text (e.g. file.txt) or gzipped (e.g. file.txt.gz) text file with one sentence in each line.

Pass the KenLM build directory to the script.

In [ ]:
%cd /content/DeepSpeech/data/lm
! python3 generate_lm.py --input_txt europarl-v7.nl-en.nl --output_dir . \
  --top_k 500000 --kenlm_bins /content/kenlm/build/bin/ \
  --arpa_order 5 --max_arpa_memory "85%" --arpa_prune "0|0|1" \
  --binary_a_bits 255 --binary_q_bits 8 --binary_type trie

Afterwards you can use `generate_package.py` to generate the scorer package using the `lm.binary` and `vocab-500000.txt` files:

In [ ]:
%cd /content/DeepSpeech/data/lm/
! python3 ./generate_package.py --alphabet /content/nl/alphabet.txt --lm lm.binary --vocab vocab-500000.txt \
  --package kenlm.scorer --default_alpha 0.931289039105002 --default_beta 1.1834137581510284

Note that we have a `lm_optimizer.py` script which can be used to find good default values for alpha and beta. To use it, you must first generate a package with any value set for default alpha and beta flags. For this step, it doesn’t matter what values you use, as they’ll be overridden by `lm_optimizer.py`. Then, use `lm_optimizer.py` with this scorer file to find good alpha and beta values. Finally, use `generate_package.py` again, this time with the new values.

*// NOTE: I coudln't get `lm_optimizer.py` to run, but the scorer was good enough to move on to the next step.*

In [ ]:
%cd /content/DeepSpeech/data/lm/
! python3 ../../lm_optimizer.py \
  --test_files /content/nl/clips/validated.csv \
  --checkpoint_dir /content/checkpoint-lm \
  --load_evaluate init \
  --scorer kenlm.scorer \
  --alphabet_config_path /content/nl/alphabet.txt

###Training a Model
*Taken from [DeepSpeech documentation - Training a Model](https://deepspeech.readthedocs.io/en/v0.7.4/TRAINING.html#training-a-model)*

The central (Python) script is DeepSpeech.py in the project’s root directory. For its list of command line options, you can call:

```python3 DeepSpeech.py --helpfull```

In [ ]:
%cd /content/DeepSpeech/
! python3 DeepSpeech.py \
  --train_files /content/nl/cv-corpus-5-2020-06-22/nl/clips/train.csv \
  --dev_files /content/nl/cv-corpus-5-2020-06-22/nl/clips/dev.csv \
  --test_files /content/nl/cv-corpus-5-2020-06-22/nl/clips/test.csv \
  --train_batch_size 1 \
  --test_batch_size 1 \
  --n_hidden 100 \
  --epochs 100 \
  --checkpoint_dir /content/model_checkpoints/ \
  --export_dir /content/models/ \
  --export_file_name 'ftmodel.pb' \
  --alphabet_config_path ../nl/alphabet.txt \
  --scorer /data/lm/kenlm.scorer

####Making a mmap-able model for inference

The `output_graph.pb` model file generated in the above step will be loaded in memory to be dealt with when running inference. This will result in extra loading time and memory consumption. One way to avoid this is to directly read data from the disk.

TensorFlow has tooling to achieve this: it requires building the target `//tensorflow/contrib/util:convert_graphdef_memmapped_format` (binaries are produced by our TaskCluster for some systems including Linux/amd64 and macOS/amd64), use `util/taskcluster.py` tool to download:

In [ ]:
%cd /content/DeepSpeech/
! python3 util/taskcluster.py --source tensorflow --artifact convert_graphdef_memmapped_format --branch r1.15 --target .

Producing a mmap-able model is as simple as:

In [ ]:
! ./convert_graphdef_memmapped_format --in_graph=/content/model/output_graph.pb --out_graph=/content/model/output_graph.pbmm

Upon sucessfull run, it should report about conversion of a non-zero number of nodes. If it reports converting `0` nodes, something is wrong: make sure your model is a frozen one, and that you have not applied any incompatible changes (this includes `quantize_weights`).

## Using a Trained Model
*Taken from [DeepSpeech documentation - Using a Pre-trained Model](https://deepspeech.readthedocs.io/en/v0.7.4/USING.html#usage-docs)*

*Now that we've trained a model, let's use it!*

## Installing DeepSpeech Python bindings

Once your environment has been set-up and loaded, you can use `pip3` to manage packages locally. On a fresh setup of the `virtualenv`, you will have to install the DeepSpeech wheel. You can check if `deepspeech` is already installed with `pip3 list`.

To perform the installation, just use `pip3` as such:

In [ ]:
! pip3 install deepspeech-gpu

*Using the model and scorer we created on one of the Common Voice test files. "ik ben tegen de doodstraf"*

In [ ]:
! deepspeech --model /content/model/output_graph.pbmm --audio /content/nl/cv-corpus-5-2020-06-22/nl/clips/common_voice_nl_19505086.wav --scorer /content/DeepSpeech/data/lm/kenlm.scorer

*Let's try the model on a new audio file we haven't used yet! Here's a YouTube clip saying 'Stroopwafel'.*

In [ ]:
%cd /content
! pip install youtube-dl
yt="https://www.youtube.com/watch?v=oXcvvuO1C7c"
! youtube-dl --extract-audio --audio-format wav {yt}

In [ ]:
from IPython.display import Audio
Audio("/content/How to pronounce the word stroopwafel - pronunciation_prononciation_pronunciación-oXcvvuO1C7c.wav")

In [ ]:
! deepspeech --model /content/model/output_graph.pbmm --audio /content/How\ to\ pronounce\ the\ word\ stroopwafel\ -\ pronunciation_prononciation_pronunciación-oXcvvuO1C7c.wav --scorer /content/DeepSpeech/data/lm/kenlm.scorer

*Not the most accurate model, but this is an example of what you can build using DeepSpeech, Common Voice, the Europarl corpus, and the free GPUs and space available on Google Colab.*